Prep the 

In [15]:
from pathlib import Path

import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd
from dask import delayed
from dask.distributed import Client

In [2]:
client = Client()

In [3]:
payroll_path = Path("data/nyc_payroll.csv")

# fiscal_year_cat = pd.CategoricalDtype(np.arange(2014, 2020), ordered=True)

pay_dtypes = {
    'fiscal_year': "int16",
    'payroll_number': None,
    'agency_name': 'category',
    'last_name': None,
    'first_name': None,
    'mid_init': None,
    'agency_start_date': object,
    'work_location_borough': 'category',
    'title_description': 'category',
    'leave_status_as_of_june_30': 'category',
    'base_salary': 'float64',
    'pay_basis': 'category',
    'regular_hours': 'float64',
    'regular_gross_paid': 'float64',
    'ot_hours': 'float64',
    'total_ot_paid': 'float64',
    'total_other_pay': 'float64',
    }

date_cols = ['agency_start_date']

all_cols = list(k for k,v in pay_dtypes.items() if v is not None)

In [ ]:
# pd.read_csv

In [4]:
pay = dd.read_csv(
    str(payroll_path), 
    dtype=pay_dtypes,
    usecols=all_cols
).persist()

In [5]:
def convert_stdt(df):
    return df.assign(
        agency_start_date=pd.to_datetime(
            df["agency_start_date"], errors="coerce", format="%m/%d/%Y")
        )

pay2 = pay.map_partitions(convert_stdt).persist()

In [6]:
years = sorted(pay2.fiscal_year.unique().compute())
years

[2014, 2015, 2016, 2017, 2018, 2019]

In [19]:
tasks = []
for year in years:
    yr = pay2.loc[pay2.fiscal_year == year, :]
    yr2 = yr.sample(frac=0.05)
    yr_out = yr2.to_csv(f"data/payroll-small/payroll_{year}.csv", single_file=True, index=False, compute=False)
    tasks.append(yr_out)
    
dask.compute(*tasks)

([None], [None], [None], [None], [None], [None])

In [ ]:
pd.DataFrame.to_csv()

In [ ]:
pay3 = pay2.set_index("fiscal_year").repartition(freq="1Y").persist()

In [ ]:
pay3.divisions

In [ ]:
pay3.npartitions

In [ ]:
pay4 = pay2.map_partitions(
    lambda df: df.sample(frac=.05)).persist()

In [ ]:
pay4.to_parquet("data/payroll-small/payroll.parq", compression="zstd")

In [ ]:
def convert_stdt(df):
    return df.assign(agency_start_date=pd.to_datetime(
        df["agency_start_date"], errors="coerce", format="%m/%d/%Y"))

pay = pay.map_partitions(convert_stdt).persist()

In [ ]:
pay = pay.persist()